In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210102.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice]
0,12/29/2020 20:24:05,Leaving Las Vegas (1995),In the Company of Men (1997),Chevalier (2016),Love and Mercy (2014),Starred Up (2013),Vanishing Point (1971),I AM NOT A GOOD DOCTOR.,LESBIANS!,OOPS ALL FRANCES MCDORMAND,RETURN TO DADDY CITY,THEY GROW UP SO FAST,LATE EIGHTIES,NaN
1,12/29/2020 20:34:26,Leaving Las Vegas (1995),Vanishing Point (1971),Starred Up (2013),Love and Mercy (2014),Chevalier (2016),In the Company of Men (1997),[Default to Nicolas Cage],LATE EIGHTIES,I AM NOT A GOOD DOCTOR.,RETURN TO DADDY CITY,THEY GROW UP SO FAST,OOPS ALL FRANCES MCDORMAND,LESBIANS!
2,12/29/2020 20:37:40,In the Company of Men (1997),Starred Up (2013),Vanishing Point (1971),Leaving Las Vegas (1995),Love and Mercy (2014),Chevalier (2016),LESBIANS!,LATE EIGHTIES,RETURN TO DADDY CITY,THEY GROW UP SO FAST,[Default to Nicolas Cage],OOPS ALL FRANCES MCDORMAND,I AM NOT A GOOD DOCTOR.
3,12/29/2020 21:39:56,Chevalier (2016),Leaving Las Vegas (1995),Love and Mercy (2014),Starred Up (2013),Vanishing Point (1971),In the Company of Men (1997),I AM NOT A GOOD DOCTOR.,RETURN TO DADDY CITY,LESBIANS!,OOPS ALL FRANCES MCDORMAND,THEY GROW UP SO FAST,LATE EIGHTIES,[Default to Nicolas Cage]
4,12/29/2020 21:41:47,Chevalier (2016),Leaving Las Vegas (1995),In the Company of Men (1997),Love and Mercy (2014),Starred Up (2013),Vanishing Point (1971),OOPS ALL FRANCES MCDORMAND,RETURN TO DADDY CITY,I AM NOT A GOOD DOCTOR.,LESBIANS!,LATE EIGHTIES,[Default to Nicolas Cage],THEY GROW UP SO FAST
5,12/30/2020 4:51:01,In the Company of Men (1997),Chevalier (2016),Leaving Las Vegas (1995),Vanishing Point (1971),Starred Up (2013),Love and Mercy (2014),OOPS ALL FRANCES MCDORMAND,LESBIANS!,[Default to Nicolas Cage],LATE EIGHTIES,I AM NOT A GOOD DOCTOR.,THEY GROW UP SO FAST,RETURN TO DADDY CITY
6,12/30/2020 11:17:33,Leaving Las Vegas (1995),Chevalier (2016),In the Company of Men (1997),Vanishing Point (1971),Starred Up (2013),Love and Mercy (2014),LESBIANS!,OOPS ALL FRANCES MCDORMAND,RETURN TO DADDY CITY,I AM NOT A GOOD DOCTOR.,LATE EIGHTIES,THEY GROW UP SO FAST,[Default to Nicolas Cage]
7,12/30/2020 14:30:50,Vanishing Point (1971),Chevalier (2016),Starred Up (2013),Love and Mercy (2014),NaN,NaN,LATE EIGHTIES,OOPS ALL FRANCES MCDORMAND,I AM NOT A GOOD DOCTOR.,THEY GROW UP SO FAST,[Default to Nicolas Cage],NaN,NaN
8,12/30/2020 14:43:00,Chevalier (2016),Vanishing Point (1971),Love and Mercy (2014),Leaving Las Vegas (1995),Starred Up (2013),In the Company of Men (1997),THEY GROW UP SO FAST,OOPS ALL FRANCES MCDORMAND,RETURN TO DADDY CITY,I AM NOT A GOOD DOCTOR.,LATE EIGHTIES,LESBIANS!,NaN
9,12/31/2020 10:11:02,Chevalier (2016),Leaving Las Vegas (1995),In the Company of Men (1997),Vanishing Point (1971),Love and Mercy (2014),Starred Up (2013),I AM NOT A GOOD DOCTOR.,RETURN TO DADDY CITY,[Default to Nicolas Cage],THEY GROW UP SO FAST,LESBIANS!,OOPS ALL FRANCES MCDORMAND,LATE EIGHTIES


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Leaving Las Vegas (1995)', 'In the Company of Men (1997)',
       'Chevalier (2016)', 'Love and Mercy (2014)', 'Starred Up (2013)',
       'Vanishing Point (1971)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Leaving Las Vegas (1995)': <Candidate('Leaving Las Vegas (1995)')>,
 'In the Company of Men (1997)': <Candidate('In the Company of Men (1997)')>,
 'Chevalier (2016)': <Candidate('Chevalier (2016)')>,
 'Love and Mercy (2014)': <Candidate('Love and Mercy (2014)')>,
 'Starred Up (2013)': <Candidate('Starred Up (2013)')>,
 'Vanishing Point (1971)': <Candidate('Vanishing Point (1971)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Leaving Las Vegas (1995), In the Company of Men (1997), Chevalier (2016), Love and Mercy (2014), Starred Up (2013), Vanishing Point (1971))>,
 <Ballot(Leaving Las Vegas (1995), Vanishing Point (1971), Starred Up (2013), Love and Mercy (2014), Chevalier (2016), In the Company of Men (1997))>,
 <Ballot(In the Company of Men (1997), Starred Up (2013), Vanishing Point (1971), Leaving Las Vegas (1995), Love and Mercy (2014), Chevalier (2016))>,
 <Ballot(Chevalier (2016), Leaving Las Vegas (1995), Love and Mercy (2014), Starred Up (2013), Vanishing Point (1971), In the Company of Men (1997))>,
 <Ballot(Chevalier (2016), Leaving Las Vegas (1995), In the Company of Men (1997), Love and Mercy (2014), Starred Up (2013), Vanishing Point (1971))>,
 <Ballot(In the Company of Men (1997), Chevalier (2016), Leaving Las Vegas (1995), Vanishing Point (1971), Starred Up (2013), Love and Mercy (2014))>,
 <Ballot(Leaving Las Vegas (1995), Chevalier (2016), In the Company of Men (1997), Vanishing P

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
Chevalier (2016)                    4  Hopeful
Leaving Las Vegas (1995)            3  Hopeful
In the Company of Men (1997)        2  Rejected
Vanishing Point (1971)              1  Rejected
Starred Up (2013)                   0  Rejected
Love and Mercy (2014)               0  Rejected

FINAL RESULT
Candidate                       Votes  Status
----------------------------  -------  --------
Chevalier (2016)                    6  Elected
Leaving Las Vegas (1995)            4  Rejected
In the Company of Men (1997)        0  Rejected
Vanishing Point (1971)              0  Rejected
Starred Up (2013)                   0  Rejected
Love and Mercy (2014)               0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                     Votes  Status
--------------------------  -------  --------
I AM NOT A GOOD DOCTOR.           3  Hopeful
OOPS ALL FRANCES MCDORMAND        2  Hopeful
LESBIANS!                         2  Hopeful
LATE EIGHTIES                     1  Hopeful
[Default to Nicolas Cage]         1  Hopeful
THEY GROW UP SO FAST              1  Rejected
RETURN TO DADDY CITY              0  Rejected

ROUND 2
Candidate                     Votes  Status
--------------------------  -------  --------
I AM NOT A GOOD DOCTOR.           3  Hopeful
OOPS ALL FRANCES MCDORMAND        3  Hopeful
LESBIANS!                         2  Hopeful
LATE EIGHTIES                     1  Rejected
[Default to Nicolas Cage]         1  Rejected
THEY GROW UP SO FAST              0  Rejected
RETURN TO DADDY CITY              0  Rejected

ROUND 3
Candidate                     Votes  Status
--------------------------  -------  --------
I AM NOT A GOOD DOCTOR.           4  Hopeful
OOPS ALL FRANCES MCDORM